# Волны московской реновации, промежуточные итоги 08.2024

In [2]:
import pandas as pd
import numpy as np
import requests
from tabula import read_pdf
import warnings
warnings.filterwarnings('ignore')

#### Данные обо всех домах из фонда реновации

In [2]:
# response_fr = requests.get("https://fr.mos.ru/pokupka-nedvizhimosti-dlya-vseh/ajax.php?status[]=OLD&pagesize=100000")
# df_fr = pd.DataFrame.from_dict(response_fr.json()["objects"]["items"])
# df_fr["longitude"] = df_fr.coords.apply(lambda x: float(x[1]))
# df_fr["latitude"] = df_fr.coords.apply(lambda x: float(x[0]))

# df_fr.to_pickle("fr.pkl")

In [3]:
df_fr = pd.read_pickle("fr.pkl")

In [4]:
df_fr.head(1)

,id,name,code,district,coords,status_code,polygon,county,ext_status,infocenter,metro,finishing_code,longitude,latitude
0,32698,"1-й Ботанический проезд, дом 1",1-j-botanicheskij-proezd-dom-1,2556,"[55.8502, 37.634]",OLD,"POLYGON((55.8499893843668 37.6338654756546,55....",2455,NaN,NaN,NaN,NaN,37.634,55.8502


#### Данные обо всех домах из mos.ru, они устаревшие и не полные данные (полные взяли у фонда ренвации), нужны для сопосоталения с unom

In [5]:
# response_old = requests.get("https://www.mos.ru/altmosprx/api/1/renovation/bounded_addresses?full&p1=0,0&p2=1000,1000")
# df_old = pd.DataFrame.from_dict(response_old.json())
# df_old["coords"] = df_old.center.apply(lambda x: x["coordinates"])
# df_old["name"] = df_old.card_fields.apply(lambda x: x["name"])
# df_old["district_name"] = df_old.card_fields.apply(lambda x: x["district_name"])
# df_old["area_name"] = df_old.card_fields.apply(lambda x: x["area_name"])
# df_old["result"] = df_old.card_fields.apply(lambda x: x["result"])
# df_old["longitude"] = df_old.coords.apply(lambda x: x[1])
# df_old["latitude"] = df_old.coords.apply(lambda x: x[0])

# df_old.to_pickle("old.pkl")

In [6]:
df_old = pd.read_pickle("old.pkl")

In [7]:
df_old.head(1)

,unom,center,polygon,card_fields,coords,name,district_name,area_name,result,longitude,latitude
0,28512,"{'type': 'Point', 'coordinates': [55.846440424...","{'type': 'Polygon', 'coordinates': [[[55.84685...","{'name': 'проезд Черепановых, дом 74', 'distri...","[55.8464404248, 37.5561827096]","проезд Черепановых, дом 74",Коптево,САО,{'protocol_res': '1'},37.556183,55.84644


#### Парсим первый приказ об этапах реновации (1.pdf), только его и можно спарсить, в нём некоторые дома совпдают по unom, для этого и нужны дынне с mos.ru

In [8]:
# def parse_pdf_table(pages, pdf_file='1.pdf'):
#     df = read_pdf(pdf_file, pages=pages, output_format = "dataframe", pandas_options={'header':None})[0]
    
#     # удаляем не нужные строки
#     df = df[~(df.iloc[:,0] == 'No п/п')]
    
#     df['unom'] = df.iloc[:,-1].combine_first(df.iloc[:,-2])
    
#     # оставляем только нужные колонки
#     df = df.iloc[:,[1, 2, 3, -1]]
#     df.columns = ['AO', 'district', 'description', 'unom']
    
#     return df

# wave_1 = parse_pdf_table('3-29') # 2020 - 2024
# wave_1['wave'] = "1"

# wave_2 = parse_pdf_table('30-76') # 2025 - 2028
# wave_2['wave'] = "2"

# wave_3 = parse_pdf_table('77-128') # 2029 - 2032
# wave_3['wave'] = "3"

# unknown = parse_pdf_table('129-148')
# unknown['wave'] = np.nan

# df_wave = pd.concat([wave_1, wave_2, wave_3, unknown], ignore_index=True)
# df_wave.to_pickle("wave.pkl")

In [9]:
df_wave = pd.read_pickle("wave.pkl")

In [10]:
df_wave.head(1)

,AO,district,description,unom,wave
0,ЦАО,Басманный,"Бакунинская ул., д.49 c.4",1316,1


#### Сопосотовляем, что можем по unom, чтобы определить волну, округ и район.

In [11]:
df_fr["key"] = df_fr.longitude.round(4).astype(str) + "_" + df_fr.latitude.round(4).astype(str)
df_old["key"] = df_old.longitude.round(4).astype(str) + "_" + df_old.latitude.round(4).astype(str)

df = df_fr.merge(df_old[["key", "unom", "area_name", "district_name"]], on="key", how='left')

In [12]:
df = df.merge(df_wave[["unom", "wave"]], on="unom", how='left')

In [13]:
mapper_ao = df[["county", "area_name"]].drop_duplicates().dropna()

In [14]:
ao = dict(zip(mapper_ao.county, mapper_ao.area_name))

In [15]:
ao

{2455: 'СВАО',
 2450: 'СЗАО',
 2457: 'ЮВАО',
 2454: 'ЦАО',
 2452: 'САО',
 2448: 'ЗАО',
 2451: 'ЮАО',
 2456: 'ВАО',
 2453: 'ЮЗАО',
 2449: 'ЗелАО',
 2459: 'ТиНАО',
 41262: 'ТиНАО'}

In [16]:
df.area_name = df.county.map(ao)

In [17]:
df.area_name.isna().any()

False

In [18]:
mapper_district = df[["district", "district_name"]].drop_duplicates().dropna()

In [19]:
district = dict(zip(mapper_district.district, mapper_district.district_name))

In [20]:
# Добавляем неопределившийся район
district[2497] = "Молжаниновский"

In [21]:
df.district_name = df.district.map(district)

In [22]:
df.district_name.isna().any()

False

#### 825 домов придётся разметить вручную

In [23]:
df_unmark = df[df.wave.isna()]

In [24]:
df_unmark = df_unmark[["area_name", "district_name", "name", "id", "wave"]].sort_values(by=["area_name", "district_name", "name"])

In [25]:
df_unmark.shape

(825, 5)

In [26]:
# Для разметки вручную из второго и третьего приказов (2.pdf и 3.pdf), кое-что из первого (1.pdf).
df_unmark.to_csv("unmark.csv", index=False)

In [27]:
# Рзмеченные вручную этапы реновации для 825 домов!
hand_marked = pd.read_csv("hand_marked.csv")
hand_marked.id = hand_marked.id.astype(str)
hand_marked.wave = hand_marked.wave.astype(str)
hand_marked = hand_marked.rename(columns={"wave": "wave_h"})

In [28]:
hand_marked.head(5)

,area_name,district_name,name,id,wave_h
0,ВАО,Богородское,"4-я Гражданская улица, дом 43, корпус 1",40118,3
1,ВАО,Восточный,"Главная улица, дом 15А",31493,2
2,ВАО,Восточный,"посёлок Акулово, дом 10",33323,3
3,ВАО,Восточный,"посёлок Акулово, дом 11",33325,3
4,ВАО,Восточный,"посёлок Акулово, дом 13",31491,3


In [29]:
df = df.merge(hand_marked[["id", "wave_h"]], on="id", how='left')

In [30]:
df.wave = df.wave.fillna(df.wave_h)

In [31]:
df.wave.isna().any()

False

#### Удаляем лишнее, выправляем данные

In [32]:
df = df[["id", "area_name", "district_name", "name", "wave", "ext_status", "longitude", "latitude"]]

In [33]:
df.head()

,id,area_name,district_name,name,wave,ext_status,longitude,latitude
0,32698,СВАО,Свиблово,"1-й Ботанический проезд, дом 1",2,NaN,37.6340,55.8502
1,32700,СВАО,Свиблово,"1-й Ботанический проезд, дом 3",3,NaN,37.6343,55.8510
2,29325,СВАО,Свиблово,"1-й Ботанический проезд, дом 3А",2,NaN,37.6336,55.8511
3,29786,СЗАО,Щукино,"1-й Волоколамский проезд, дом 7, корпус 1",1,destroyed,37.4918,55.8008
4,29788,СЗАО,Щукино,"1-й Волоколамский проезд, дом 7, корпус 2",1,destroyed,37.4911,55.8007


In [34]:
df.ext_status.unique()

array([nan, 'destroyed', 'settling', 'settled'], dtype=object)

In [35]:
df[df.ext_status == "settled"]

,id,area_name,district_name,name,wave,ext_status,longitude,latitude
4816,29958,ЮВАО,Кузьминки,"улица Юных Ленинцев, дом 44, корпус 2",2,settled,37.7599,55.6999


In [36]:
# settled тоже самое, что и снесено
df.loc[df.ext_status == "settled", "ext_status"] = "destroyed"

In [37]:
# nan - значит, что реновация не стартовала, для удобства заменим на not_started
df.ext_status = df.ext_status.fillna("not_started")

In [38]:
df.ext_status.unique()

array(['not_started', 'destroyed', 'settling'], dtype=object)

In [39]:
# settling - Переселение производится прямо сейчас
# destroyed - Дом снесён
# not_started - Реновация ещё не началась

#### На полученных данных теперь можно оценить результаты реновации!

In [3]:
# df.to_pickle("final.pkl")
df = pd.read_pickle("final.pkl")

In [4]:
n = len(df)
print("Количество домов входящих в реновацию, всего:", n)

Количество домов входящих в реновацию, всего: 5176


In [5]:
first = np.sum(df.wave == "1")
second = np.sum(df.wave == "2")
third = np.sum(df.wave == "3")

print(f"Первый этап 2020 — 2024гг., в него вошло {first} домов, доля от общего числа {first / n * 100:.3}%")
print(f"Второй этап 2025 — 2028гг., в него вошло {second} домов, доля от общего числа {second / n * 100:.3}%")
print(f"Третий этап 2029 — 2032гг., в него вошло {third} домов, доля от общего числа {third / n * 100:.3}%")

Первый этап 2020 — 2024гг., в него вошло 1193 домов, доля от общего числа 23.0%
Второй этап 2025 — 2028гг., в него вошло 1888 домов, доля от общего числа 36.5%
Третий этап 2029 — 2032гг., в него вошло 2095 домов, доля от общего числа 40.5%


На сегодняшний день первая волна почти уже закончилась, до её завершения осталось чуть больше трёх месяцев. Значит, все дома первой волны, которые ещё не снесены, должны быть в процессе переселения как минимум, посмотрим, на сколько на данный момент она выполнена.

In [6]:
first_part = np.sum((df.wave == "1") & (df.ext_status != "not_started"))
first_n = np.sum(df.wave == "1")
print(f"Первой волной реновацией был затронут {first_part} дом")
print(f"Первая волна реновации выполнена на данный момент на {first_part / first_n * 100:.3}%")

Первой волной реновацией был затронут 921 дом
Первая волна реновации выполнена на данный момент на 77.2%


Теперь можно рассмотреть результаты реновации в общем по количеству снесённых домов не зависимо от волны.

In [9]:
destroed = np.sum(df.ext_status == "destroyed")
print(f"Из них уже снесенно: {destroed} домов")
print(f"Доля cнесённых от общего числа: {destroed / n * 100:.3}%")

Из них уже снесенно: 461 домов
Доля cнесённых от общего числа: 8.91%


In [10]:
settling = np.sum(df.ext_status == "settling")
print(f"Количество домов, которые переселяются уже сейчас: {settling} домов")
print(f"Доля переселяемых домов от общего числа: {settling / n * 100:.3}%")

Количество домов, которые переселяются уже сейчас: 565 домов
Доля переселяемых домов от общего числа: 10.9%


Рассмотрим подробнее в общем, сколько должно быть снесено домов и что есть по факту.

In [11]:
print(f"Количество домов активно затронутых реновацией не зависимо от волны {destroed + settling} домов")
print(f"Завершённость реновации в целом не зависимо от волны: {(destroed + settling) / n * 100:.3}%")

Количество домов активно затронутых реновацией не зависимо от волны 1026 домов
Завершённость реновации в целом не зависимо от волны: 19.8%


In [14]:
print(f"Количество домов первной волны активно не затронутых реновацией {first_n - first_part} дома")
print(f"От общего числа домов: {(first_n - first_part) / n * 100:.3}%")

Количество домов первной волны активно не затронутых реновацией 272 домов
От общего числа домов: 5.26%


До конца первой волны остались считанные месяцы, но для её полной реалзиации необходимо обхватить ещё 272 дома, что приводит к задолженности в 5.26% на данный момент и к началу 2025 года эта цифра скорее всего изменится не сильно, всё строительство или переселение - это не быстрые процессы

Вторая волна ещё больше первой по количеству домов, а треться - самая большая, так что отставание скорее всего так и будет нарастать. Хотя кончено как знать, сейчас переселяется домов больше, чем было снесено за прошедшее время программы. Однако, реновация уже отстаёт от плана, так что она получится только если в каких-то местах будет форсирована.

Исоходя из этого, можно оценить реновацию с другой строны - без учёта волн, ведь в некторых районах уже затрагиваются дома второй или даже третьей волны. Таким образом в результате первой волны в прогамме должно было активно поучаствовать 23.0% от всех домов, а на данный момент их число равно 19.8%, другими словами 23.0-19.8 = 3.2% можно скзать, что это минимальная оценка задолженности.

Исходя из этого получаем оптимистичную оценку задолженности 3.2% и негативную в 5.26%. Эти цифры едвали сильно изменятся за оставшееся время до конца первой волны. Учитывая насколько текущая программа реновации сложная и большая, задолженность от 3.2% до 5.26% выглядит просто фантастически маленькой.

Рассмотрим теперь результаты по каждому округу, а затем и по каждому району.

In [22]:
def stat_func(df):
    first_n = np.sum(df.wave == "1")
    first_part = np.sum((df.wave == "1") & (df.ext_status != "not_started"))
    first_complete = first_part / first_n * 100

    n = len(df)
    plan = first_n / n * 100
    renovated = np.sum(df.ext_status != "not_started") / n * 100
    forced = renovated - plan

    
    return pd.Series(
        {
            "first_complete": round(first_complete, 2), 
            "plan": round(plan, 2),
            "renovated": round(renovated, 2),
            "forced": round(forced, 2)
        }
    )

In [24]:
ao_stat = (
    df.groupby("area_name")
     .apply(stat_func)
     .reset_index()
     .sort_values(by=["first_complete", "forced", "area_name"], ascending=False)
     .rename(columns={
         "area_name": "Округ", 
         "first_complete":"Завершение первой волны %",
         "plan": "По плану от общего числа в округе %",
         "renovated": "Текущее от общего числа в округе %",
         "forced": "Опережение или отставание от плана %",
     })
)
ao_stat.index = np.arange(1, len(ao_stat) + 1)
ao_stat.to_excel("ao_stat.xlsx") 
print(ao_stat)

    Округ  Завершение первой волны %  По плану от общего числа в округе %  \
1   ЗелАО                     100.00                                35.29   
2    ЮЗАО                      85.98                                20.58   
3    СВАО                      82.95                                17.64   
4     ЦАО                      82.93                                37.27   
5     ЗАО                      82.17                                28.09   
6    ЮВАО                      80.66                                22.13   
7     ЮАО                      78.02                                24.07   
8   ТиНАО                      76.81                                55.42   
9    СЗАО                      74.51                                11.81   
10    САО                      72.12                                20.19   
11    ВАО                      65.02                                21.00   

    Текущее от общего числа в округе %  Опережение или отставание от плана 

In [26]:
district_stat = (
    df.groupby(["area_name", "district_name"])
     .apply(stat_func)
     .reset_index()
     .sort_values(by=["area_name", "first_complete", "forced"], ascending=False)
     .rename(columns={
         "area_name": "Округ",
         "district_name": "Район",
         "first_complete":"Завершение первой волны %",
         "plan": "По плану от общего числа в районе %",
         "renovated": "Текущее от общего числа в районе %",
         "forced": "Опережение или отставание от плана %",
     })
)
district_stat.index = np.arange(1, len(district_stat) + 1)
district_stat = district_stat.fillna("Нет домов первой волны")


district_stat.to_excel("district_stat.xlsx") 
print(district_stat.head())

  Округ            Район Завершение первой волны %  \
1  ЮЗАО         Коньково                     100.0   
2  ЮЗАО  Северное Бутово                     100.0   
3  ЮЗАО        Черёмушки                     100.0   
4  ЮЗАО     Южное Бутово                     100.0   
5  ЮЗАО          Ясенево                     100.0   

   По плану от общего числа в округе %  Текущее от общего числа в округе %  \
1                                39.02                               43.90   
2                                50.00                               50.00   
3                                 5.88                                5.88   
4                                38.46                               38.46   
5                               100.00                              100.00   

   Опережение или отставание от плана %  
1                                  4.88  
2                                  0.00  
3                                  0.00  
4                                  0.0